### Importando bibliotecas

In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns


In [2]:
df = pd.read_csv('googl_daily_prices.csv')
df.head()

,date,1. open,2. high,3. low,4. close,5. volume
0,2025-05-21,163.690,173.140,163.5600,168.56,71569839.0
1,2025-05-20,166.430,168.500,162.9000,163.98,46607656.0
2,2025-05-19,164.510,166.640,164.2200,166.54,30426097.0
3,2025-05-16,167.725,169.350,165.6200,166.19,42846925.0
4,2025-05-15,165.840,166.205,162.3732,163.96,33146669.0


In [3]:
df.shape

(5223, 6)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5223 entries, 0 to 5222
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   date       5223 non-null   object 
 1   1. open    5223 non-null   float64
 2   2. high    5223 non-null   float64
 3   3. low     5223 non-null   float64
 4   4. close   5223 non-null   float64
 5   5. volume  5223 non-null   float64
dtypes: float64(5), object(1)
memory usage: 245.0+ KB
